In [ ]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np

# ============================================
# 1️⃣ 读取图像
# ============================================
image_np = cv2.imread("../image/work4.png")

# 转换为灰度图
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# ============================================
# 2️⃣ 二值化处理
# ============================================
# cv2.threshold(src, thresh, maxval, type)
#   - src：输入灰度图像
#   - thresh：阈值（127 表示灰度分界）
#   - maxval：满足条件的像素被赋予的最大值（通常 255）
#   - type：阈值化类型
#       cv2.THRESH_BINARY      →  大于阈值为255，否则为0
#       cv2.THRESH_BINARY_INV  →  大于阈值为0，否则为255
_, image_thresh = cv2.threshold(image_gray, 127, 255, cv2.THRESH_BINARY_INV)

# ============================================
# 3️⃣ 查找轮廓
# ============================================
# cv2.findContours(image, mode, method[, offset])
# --------------------------------------------
# 参数：
#   image：输入的二值图像（非零像素视为前景）
#   mode ：轮廓检索模式
#       cv2.RETR_EXTERNAL ：仅检测最外层轮廓
#       cv2.RETR_TREE     ：建立完整的层级关系
#   method：轮廓近似方法
#       cv2.CHAIN_APPROX_SIMPLE ：压缩冗余点，仅保留拐点
# 返回：
#   contours ：列表，每个元素是一条轮廓（N×1×2 数组）
#   hierarchy：层级信息（形状为 (1, N, 4)）
contours, hierarchy = cv2.findContours(
    image_thresh,
    cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE
)

# ============================================
# 4️⃣ 计算并绘制凸包
# ============================================
# cv2.convexHull(points[, hull[, clockwise[, returnPoints]]]) → hull
# ------------------------------------------------------------
# 参数说明：
#   points：输入的轮廓点集（来自 findContours）
#   hull：可选参数，用于存储输出（通常省略）
#   clockwise：
#       - True  → 输出的点按顺时针方向排列
#       - False → 输出的点按逆时针方向排列（默认）
#   returnPoints：
#       - True  → 返回凸包上的坐标点（默认）
#       - False → 返回凸包点的索引（用于匹配轮廓点）
#
# 返回：
#   hull：凸包点的坐标集（与轮廓同类型的 N×1×2 数组）
#
# 说明：
#   凸包是包围轮廓的最小凸多边形，可用于形状分析、物体包围框计算等。
#   若想检测物体是否是凸形，可用 cv2.isContourConvex(contour)
image_hull = image_np.copy()  # 复制原图用于绘制

for i, contour in enumerate(contours):
    # 计算当前轮廓的凸包
    hull = cv2.convexHull(contour, returnPoints=True)

    # 绘制原始轮廓（蓝色）
    cv2.drawContours(image_hull, [contour], -1, (255, 0, 0), 2)

    # 绘制凸包（绿色）
    cv2.drawContours(image_hull, [hull], -1, (0, 255, 0), 2)

    # 标注编号
    x, y, w, h = cv2.boundingRect(contour)
    cv2.putText(image_hull, f"#{i}", (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# ============================================
# 5️⃣ 计算并输出相关属性
# ============================================
print(f"检测到 {len(contours)} 个轮廓")

for i, c in enumerate(contours):
    area = cv2.contourArea(c)               # 轮廓面积
    hull = cv2.convexHull(c)                # 凸包点集
    hull_area = cv2.contourArea(hull)       # 凸包面积
    solidity = float(area) / hull_area if hull_area != 0 else 0  # 凸度（轮廓面积 / 凸包面积）
    perimeter = cv2.arcLength(c, True)      # 轮廓周长
    convex = cv2.isContourConvex(c)         # 是否为凸形

    print(f"轮廓 {i}: 面积={area:.2f}, 周长={perimeter:.2f}, 凸包面积={hull_area:.2f}, 凸度={solidity:.3f}, 是否凸形={convex}")

# ============================================
# 6️⃣ 显示结果
# ============================================
cv2.imshow("Binary Image", image_thresh)
cv2.imshow("Convex Hull Detection", image_hull)
cv2.waitKey(0)
cv2.destroyAllWindows()


检测到 4 个轮廓
轮廓 0: 面积=992.00, 周长=155.66, 凸包面积=1084.00, 凸度=0.915, 是否凸形=False
轮廓 1: 面积=826.50, 周长=356.18, 凸包面积=1648.50, 凸度=0.501, 是否凸形=False
轮廓 2: 面积=3165.50, 周长=441.49, 凸包面积=4339.00, 凸度=0.730, 是否凸形=False
轮廓 3: 面积=27379.00, 周长=680.00, 凸包面积=27379.00, 凸度=1.000, 是否凸形=True
